In [1]:
import numpy as np
import pandas as pd
np.random.seed(0)


In [2]:
folds_dir = './ml-100k/'

header = ['user_id', 'item_id', 'rating', 'timestamp']   
df = pd.read_csv(f'{folds_dir}u.data', sep='\t', names=header)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
def get_all_users_and_items():
    header = ['user_id', 'item_id', 'rating', 'timestamp']   
    df = pd.read_csv(f'{folds_dir}u.data', sep='\t', names=header)
    users = df.user_id.unique()
    items = df.item_id.unique()
    return users, items
all_users_ids, all_items_ids = get_all_users_and_items()


In [4]:

def create_rating_matrix_from_raw_data(df):

    ratings = np.zeros((all_users_ids.shape[0], all_items_ids.shape[0]))

    for row in df.itertuples():
        ratings[row[1]-1][row[2]-1] = row[3]  
        
    return ratings


In [5]:
df = pd.read_csv(f'{folds_dir}u.data', sep='\t', names=header)
df.shape

ratings = create_rating_matrix_from_raw_data(df)
ratings.shape

(943, 1682)

In [6]:
import numpy as np
sparsity = float(len(np.nan_to_num(ratings).nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 6.30%


In [10]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [11]:
train, test = train_test_split(ratings)

In [ ]:
def get_5_folds(folds_dir='./ml-100k/'):    
    header = ['user_id', 'item_id', 'rating', 'timestamp']
    folds = []
    
    for i in range(5):     
        print(f'{folds_dir}u{i+1}.base')
        df_train = pd.read_csv(f'{folds_dir}u{i+1}.base', sep='\t', names=header)
        df_test = pd.read_csv(f'{folds_dir}u{i+1}.test', sep='\t', names=header)   
        
        rating_train = create_rating_matrix_from_raw_data(df_train)
        rating_test  = create_rating_matrix_from_raw_data(df_test)        
           
        folds.append((rating_train, rating_test))
    return folds
folds = get_5_folds()

for fold in folds:
    print(fold[0].shape, fold[1].shape)

In [ ]:
train_data, test_data = folds[0]
train_data.shape

In [7]:
import numpy as np
from numpy.linalg import solve
from sklearn.metrics import mean_squared_error


def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [8]:
from numpy.linalg import solve
import numpy as np

class ExplicitMF():
    def __init__(self,
                 ratings,
                 n_factors=40,
                 item_reg=0.0,
                 user_reg=0.0,
                 verbose=False):
        """
        Train a matrix factorization model to predict empty 
        entries in a matrix. The terminology assumes a 
        ratings matrix which is ~ user x item

        Params
        ======
        ratings : (ndarray)
            User x Item matrix with corresponding ratings

        n_factors : (int)
            Number of latent factors to use in matrix 
            factorization model

        item_reg : (float)
            Regularization term for item latent factors

        user_reg : (float)
            Regularization term for user latent factors

        verbose : (bool)
            Whether or not to printout training progress
        """

        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.item_reg = item_reg
        self.user_reg = user_reg
        self._v = verbose

    def als_step(self,
                 latent_vectors,
                 fixed_vecs,
                 ratings,
                 _lambda,
                 type='user'):
        """
        One of the two ALS steps. Solve for the latent vectors
        specified by type.
        """
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI),
                                             ratings[u, :].dot(fixed_vecs))
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda

            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI),
                                             ratings[:, i].T.dot(fixed_vecs))
        return latent_vectors

    def train(self, n_iter=10):
        """ Train model for n_iter iterations from scratch."""
        # initialize latent vectors
        self.user_vecs = np.random.random((self.n_users, self.n_factors))
        self.item_vecs = np.random.random((self.n_items, self.n_factors))

        self.partial_train(n_iter)

    def partial_train(self, n_iter):
        """ 
        Train model for n_iter iterations. Can be 
        called multiple times for further training.
        """
        ctr = 1
        while ctr <= n_iter:
            if ctr % 10 == 0 and self._v:
                print('\tcurrent iteration: {}'.format(ctr))
            self.user_vecs = self.als_step(self.user_vecs,
                                           self.item_vecs,
                                           self.ratings,
                                           self.user_reg,
                                           type='user')
            self.item_vecs = self.als_step(self.item_vecs,
                                           self.user_vecs,
                                           self.ratings,
                                           self.item_reg,
                                           type='item')
            ctr += 1

    def predict_all(self):
        """ Predict ratings for every user and item. """
        predictions = np.zeros((self.user_vecs.shape[0],
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)

        return predictions

    def predict(self, u, i):
        """ Single user and item prediction. """
        return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)

    def calculate_learning_curve(self, iter_array, test):
        """
        Keep track of MSE as a function of training iterations.

        Params
        ======
        iter_array : (list)
            List of numbers of iterations to train for each step of 
            the learning curve. e.g. [1, 5, 10, 20]
        test : (2D ndarray)
            Testing dataset (assumed to be user x item).

        The function creates two new class attributes:

        train_mse : (list)
            Training data MSE values for each value of iter_array
        test_mse : (list)
            Test data MSE values for each value of iter_array
        """
        iter_array.sort()
        self.train_mse = []
        self.test_mse = []
        iter_diff = 0
        for (i, n_iter) in enumerate(iter_array):
            if self._v:
                print('Iteration: {}'.format(n_iter))
            if i == 0:
                self.train(n_iter - iter_diff)
            else:
                self.partial_train(n_iter - iter_diff)

            predictions = self.predict_all()

            self.train_mse += [get_mse(predictions, self.ratings)]
            self.test_mse += [get_mse(predictions, test)]
            if self._v:
                print('Train mse: ' + str(self.train_mse[-1]))
                print('Test mse: ' + str(self.test_mse[-1]))
            iter_diff = n_iter

In [13]:
import numpy as np
from numpy.linalg import solve
from sklearn.metrics import mean_squared_error


def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)


class MatrixFactorizationALS:
    def __init__(self,
                 ratings,
                 k_factors=40,
                 item_reg=0.0,
                 user_reg=0.0,
                 verbose=False):
        self.ratings = ratings
        self.users_count = users_count
        self.items_count = items_count        
        self.k_factors = k_factors
        self.item_reg = item_reg
        self.user_reg = user_reg
        self._v = verbose
        self.curr_iter = 1

    def calculate_learning_curve(self, iter_array, test):
        iter_array.sort()
        self.init_for_train()
        training_mse, test_mse = [], []
        
        for n_iter in iter_array:            
            self.print_verbos(f'k={self.k_factors}, '\
                              f'iterations={n_iter}, '\
                              f'item_reg={self.item_reg}, '\
                              f'user_reg={self.user_reg}')
            self.train(n_iter)
            predictions = self.get_prediction_matrix()
            self.evaluate(predictions, training_mse, test_mse, test)  
        print('======================')
        return training_mse, test_mse
    
    def als_step_user(self):
        YTY = self.item_vecs.T.dot(self.item_vecs)
        lambdaI = np.eye(YTY.shape[0]) * self.user_reg

        for u in range(self.user_vecs.shape[0]):
            self.user_vecs[u, :] = solve((YTY + lambdaI), self.ratings[u, :].dot(self.item_vecs))

    def als_step_item(self):
        XTX = self.user_vecs.T.dot(self.user_vecs)
        lambdaI = np.eye(XTX.shape[0]) * self.item_reg

        for i in range(self.item_vecs.shape[0]):
            self.item_vecs[i, :] = solve((XTX + lambdaI), self.ratings[:, i].T.dot(self.user_vecs))

    def init_for_train(self:
        # initialize latent vectors
        self.user_vecs = np.random.normal(scale=1. / self.k_factors,
                                          size=(self.users_count, self.k_factors))
        self.item_vecs = np.random.normal(scale=1. / self.k_factors,
                                          size=(self.items_count, self.k_factors))
        print(f'user_vecs: {self.user_vecs.shape}, item_vecs: {self.item_vecs.shape}')


    def train(selfת n_iter):
        """ 
        Train model for n_iter iterations. 
        Can be called multiple times for further training.
        """
        while self.curr_iter <= n_iter:
            if self.curr_iter % 10 == 0 and self._v:
                print(f'\tStill running...')
                print(f'\tCurrent iteration: {self.curr_iter}')
                
            self.als_step_user()
            self.als_step_item()
            self.curr_iter += 1
            
    def predict(self, u, i):
        return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)

    def predict_all(self):
        predictions = np.zeros((self.user_vecs.shape[0],
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
        return predictions
                       
    def evaluate(self, predictions, training_mse, test_mse, test):
        training_mse.append(get_mse(predictions, self.ratings))
        test_mse.append(get_mse(predictions, test))

        self.print_verbos(f'Training MSE = {training_mse[-1]}')
        self.print_verbos(f'Test MSE = {test_mse[-1]}')
                       

In [10]:
from collections import defaultdict

ohad = defaultdict(list)

for i in range(10):
    ohad[i].append(i)
    
ohad

defaultdict(list,
            {0: [0],
             1: [1],
             2: [2],
             3: [3],
             4: [4],
             5: [5],
             6: [6],
             7: [7],
             8: [8],
             9: [9]})

In [ ]:
class ExplicitMFSGD():
    def __init__(self, 
                 ratings,
                 k_factors = 40,
                 item_fact_reg = 0.0, 
                 user_fact_reg = 0.0,
                 item_bias_reg = 0.0,
                 user_bias_reg = 0.0,
                 verbose = False):
        """
        Train a matrix factorization model to predict empty entries in a matrix. 
        The terminology assumes a ratings matrix which is ~ user x item.
        
        (To avoid overfitting we use regularization)

        Params
        ======
        ratings: (ndarray)
            User x Item matrix with corresponding ratings
        
        k_factors: (int)
            Number of latent factors to use in matrix factorization model
        
        item_fact_reg: (float)
            Regularization term for item latent factors
            
        user_fact_reg: (float)
            Regularization term for user latent factors
            
        item_bias_reg: (float)
            Regularization term for item biases
        
        user_bias_reg: (float)
            Regularization term for user biases
        
        verbose: (bool)
            Whether or not to printout training progress
        """
        
        self.ratings = ratings
        self.users_count = users_count
        self.items_count = items_count
        self.k_factors = k_factors
        self.item_fact_reg = item_fact_reg
        self.user_fact_reg = user_fact_reg
        self.item_bias_reg = item_bias_reg
        self.user_bias_reg = user_bias_reg
        
        self.sample_row, self.sample_col = self.ratings.nonzero()
        self.n_samples = len(self.sample_row)
        self._v = verbose
        self.curr_iter = 1

        
    def calculate_learning_curve(self, iter_array, test, learning_rate):
        iter_array.sort()
        self.init_for_train(learning_rate)
        training_mse, test_mse = [], []
        
        for n_iter in iter_array:            
            self.print_verbose(f'k={self.k_factors}, alpha={learning_rate}, '\
                              f'iterations={n_iter}, item_fact_reg={self.item_fact_reg}, '\
                              f'user_fact_reg={self.user_fact_reg}, item_bias_reg={self.item_bias_reg}, '\
                              f'user_bias_reg={self.user_bias_reg}')
            self.train(n_iter)
            predictions = self.get_prediction_matrix()
            self.evaluate(predictions, training_mse, test_mse, test)
            
        print('======================')
        
        return training_mse, test_mse
    
    
    def init_for_train(self, learning_rate=0.1):        
        # initialize latent vectors
        # Approximate rating matrix by product of lower rank matrix
        self.user_vecs = np.random.normal(scale=1./self.k_factors, size=(self.users_count, self.k_factors))
        self.item_vecs = np.random.normal(scale=1./self.k_factors, size=(self.items_count, self.k_factors))
        
        self.learning_rate = learning_rate
        self.user_bias = np.zeros(self.users_count)
        self.item_bias = np.zeros(self.items_count)
        self.global_bias = np.mean(self.ratings[np.where(self.ratings != 0)])

    def train(self, n_iter):
        """ 
        Train model for n_iter iterations. 
        Can be called multiple times for further training.
        """
        while self.curr_iter <= n_iter:
            if self.curr_iter % 10 == 0 and n_iter > 10 and self._v:
                print(f'\tStill running...')
                print(f'\tCurrent iteration: {self.curr_iter}')
                
            self.training_indices = np.arange(self.n_samples)
            np.random.shuffle(self.training_indices)
            self.perform_sgd()
            self.curr_iter += 1
            
    def predict(self, user, item):
        """
        Single user and item prediction
        """
        biases = self.global_bias + self.user_bias[user] + self.item_bias[item]
        prediction_value = biases + self.user_vecs[user, :].dot(self.item_vecs[item, :].T)
        
        return prediction_value
    
    def perform_sgd(self):
        for idx in self.training_indices:
            user = self.sample_row[idx]
            item = self.sample_col[idx]
            prediction = self.predict(user, item)
            actual_rating = self.ratings[user, item] # get actual rating from the dataset's ratings array 
            error = actual_rating - prediction
            
            # Update biases
            self.user_bias[user] += self.learning_rate * (error - self.user_bias_reg * self.user_bias[user])
            self.item_bias[item] += self.learning_rate * (error - self.item_bias_reg * self.item_bias[item])
            
            # Update latent factors
            self.user_vecs[user, :] += self.learning_rate * (error * self.item_vecs[item, :] - self.user_fact_reg * self.user_vecs[user,:])
            self.item_vecs[item, :] += self.learning_rate * (error * self.user_vecs[user, :] - self.item_fact_reg * self.item_vecs[item,:])
    
    def get_prediction_matrix(self):
        """
        Predict ratings for every user and item
        """
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        
        for user in range(self.user_vecs.shape[0]):
            for item in range(self.item_vecs.shape[0]):
                predictions[user, item] = self.predict(user, item)
                
        return predictions
    
    def evaluate(self, predictions, training_mse, test_mse, test):
        training_mse.append(get_mse(predictions, self.ratings))
        test_mse.append(get_mse(predictions, test))

        self.print_verbose(f'Training MSE = {training_mse[-1]}')
        self.print_verbose(f'Test MSE = {test_mse[-1]}')

    def print_verbose(self, msg):
        if self._v:
            print(msg)

In [14]:
MF_ALS = MatrixFactorizationALS(train, n_factors=40, \
                    user_reg=0.0, item_reg=0.0, verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]
MF_ALS.calculate_learning_curve(iter_array, test)

Iteration: 1
user_vecs: (943, 40), item_vecs: (1682, 40)
Train mse: 5.508768969828915
Test mse: 9.803011129993122
Iteration: 2
Train mse: 4.211806356215287
Test mse: 8.651950992137149
Iteration: 5
Train mse: 3.9684131825311786
Test mse: 8.465987678112086
Iteration: 10
Train mse: 3.9338102082318858
Test mse: 8.448744189802962
Iteration: 25
	current iteration: 10
Train mse: 3.925227446759353
Test mse: 8.451516825902674
Iteration: 50
	current iteration: 10
	current iteration: 20
Train mse: 3.9242229016201344
Test mse: 8.46562199755438
Iteration: 100
	current iteration: 10
	current iteration: 20
	current iteration: 30
	current iteration: 40
	current iteration: 50
Train mse: 3.923497245580798
Test mse: 8.466999943345845


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

def plot_learning_curve(iter_array, model):
    plt.plot(iter_array, model.train_mse, \
             label='Training', linewidth=5)
    plt.plot(iter_array, model.test_mse, \
             label='Test', linewidth=5)


    plt.xticks(fontsize=16);
    plt.yticks(fontsize=16);
    plt.xlabel('iterations', fontsize=30);
    plt.ylabel('MSE', fontsize=30);
    plt.legend(loc='best', fontsize=20);

In [ ]:
plot_learning_curve(iter_array, MF_ALS)